# Conv-adapter (residual) experiments with different Factors

In [6]:
import sync_wandb_run
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import utils as pu
from plot import create_plot_with_values
from plot_pretty import create_ultimate_plotly_plot, generate_scatter_plot, generate_scatter_plot_accuracy, generate_scatter_plot_gridscore
import plotly_utils

In [7]:
entity = "raphaelmaser"  # Your W&B username or team
project = "Improved-Interpretability-and-Concepts"  # Your W&B project name
df = sync_wandb_run.load_wandb_runs()
df.count()  # Display the number of runs loaded
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

val_accuracy = "Validation Accuracy"
grid_score = "PG Score"
frozen_layers = "config_define_frozen_layers"
trainable_params = "Trainable Parameters (M)"
figsize=(10,4)


In [8]:
def rename_model_names(df):
    """Rename model names in the DataFrame for better readability."""
    df = df.replace({
        "Model": {
            "B=1_pretrained_CE_bcosified": "B=2 (Bcosified Model)",
            "B=1_pretrained_CE-parallel-residual-adapters-B=2_B=2": "B=2 Adapter",
        }
    })
    return df

In [9]:
conv_adapter = df[df['group'] == "adapter-factor"]

bcosified_model = df[df['id'] == "29kcufiq"]
conv_adapter = pd.concat([conv_adapter, bcosified_model], ignore_index=True)  # Add the Bcosified model to the DataFrame
# conv_wo_batchnorm = conv_wo_batchnorm[conv_wo_batchnorm["id"].isin(["sbpfkwct", "dv8y7o8k", "yheexwdv", "zf9oo2l6", "xka2n8rp"])]  # Filter for specific runs
# conv_wo_batchnorm = rename(conv_wo_batchnorm)  # Rename model names for better readability
conv_adapter.reset_index(drop=True, inplace=True)  # Reset index after filtering
print(f"Number of runs in layerwise_freezing group: {len(conv_adapter)}")

#Renaming
conv_adapter = pu.rename(conv_adapter).sort_values(by=val_accuracy, ascending=False)
conv_adapter = rename_model_names(conv_adapter)  # Rename model names for better readability
# conv_adapter = pu.add_trainable_layers_column(conv_adapter, frozen_layers_col=frozen_layers, new_col_name=trainable_layers)
conv_adapter = pu.rename_adapter_factor_experiments(conv_adapter)  # Rename columns for adapter factor experiments

# Some stats
for name in conv_adapter["Model"].unique(): print(name)  # Print each model name
display(conv_adapter)


Number of runs in layerwise_freezing group: 6
B=2 (Bcosified Model)
B=2 Adapter


id                  Model     state        entity  \
5  29kcufiq  B=2 (Bcosified Model)  finished  raphaelmaser   
1  3xroiask            B=2 Adapter  finished  raphaelmaser   
4  ewtdtath            B=2 Adapter  finished  raphaelmaser   
3  awu15mtb            B=2 Adapter  finished  raphaelmaser   
0  bqdqeryg            B=2 Adapter  finished  raphaelmaser   
2  wz3veosi            B=2 Adapter    failed  raphaelmaser   

                                  project            created_at  updated_at  \
5  Improved-Interpretability-and-Concepts  2025-06-30T09:25:37Z         NaN   
1  Improved-Interpretability-and-Concepts  2025-06-30T12:58:28Z         NaN   
4  Improved-Interpretability-and-Concepts  2025-06-30T14:31:25Z         NaN   
3  Improved-Interpretability-and-Concepts  2025-06-30T14:31:25Z         NaN   
0  Improved-Interpretability-and-Concepts  2025-06-30T14:41:44Z         NaN   
2  Improved-Interpretability-and-Concepts  2025-06-30T12:14:18Z         NaN   

  tags notes                                                url  \
5  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
1  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
4  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
3  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
0  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
2  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   

                 group  job_type  sweep                   synced_at  config_b  \
5  adapter-experiments       NaN    NaN  2025-07-24T10:44:04.193189       2.0   
1       adapter-factor       NaN    NaN  2025-07-24T10:44:09.305100       2.0   
4       adapter-factor       NaN    NaN  2025-07-24T10:44:11.010808       2.0   
3       adapter-factor       NaN    NaN  2025-07-24T10:44:10.175138       2.0   
0       adapter-factor       NaN    NaN  2025-07-24T10:44:11.997670       2.0   
2       adapter-factor       NaN    NaN  2025-07-24T10:44:08.601129       2.0   

   config_lr config_loss  config_seed config_tags         config_group  \
5      0.001  UniformBCE         42.0         NaN  adapter-experiments   
1      0.001  UniformBCE         42.0         NaN       adapter-factor   
4      0.001  UniformBCE         42.0         NaN       adapter-factor   
3      0.001  UniformBCE         42.0         NaN       adapter-factor   
0      0.001  UniformBCE         42.0         NaN       adapter-factor   
2      0.001  UniformBCE         42.0         NaN       adapter-factor   

              config_model  config_epochs config_adapter  config_compile  \
5  BcosResNet50_bottleneck             30            NaN            True   
1  BcosResNet50_bottleneck             30            NaN            True   
4  BcosResNet50_bottleneck             30            NaN            True   
3  BcosResNet50_bottleneck             30            NaN            True   
0  BcosResNet50_bottleneck             30            NaN            True   
2  BcosResNet50_bottleneck             30            NaN            True   

  config_dataset  config_vit_dim config_use_lora config_act_layer  \
5   TinyImagenet             NaN             NaN             relu   
1   TinyImagenet           768.0             NaN             relu   
4   TinyImagenet           768.0             NaN             relu   
3   TinyImagenet           768.0             NaN             relu   
0   TinyImagenet           768.0             NaN             relu   
2   TinyImagenet             NaN             NaN             relu   

   config_b_adapter                                   config_data_path  \
5               2.0  /home/rmaser/rmaser_improved_interpretability_...   
1               2.0  /home/rmaser/rmaser_improved_interpretability_...   
4               2.0  /home/rmaser/rmaser_improved_interpretability_...   
3               2.0  /home/rmaser/rmaser_improved_interpretability_...   
0               2.0  /home/rmaser/rmaser_improved_interpretability_...   
2 

In [10]:
# # Move legend outside the plot
# plt.tight_layout()
# plt.show()

# plt.figure(figsize=figsize)
# ax = sns.scatterplot(
#     data=conv_adapter,
#     y=val_accuracy,
#     x=trainable_params,
#     hue="Model",  # Color points by name
#     # s=100  # Make points larger
# )
# pu.add_val_to_plot(ax)
# plt.ylim(0.7, 0.90)
# # Move legend outside the plot
# plt.tight_layout()
# plt.show()

generate_scatter_plot_accuracy(
    data=conv_adapter,
    y=val_accuracy,
    x=trainable_params,
    color="Model",
    title="Conv-Adapter (Residual)", 
    scale_factor=2,
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params
    )

In [ ]:
# Let's explore the different data formats available
print("=== Current Summary Format (what you've been using) ===")
print("Shape:", df.shape)
print("Columns:", [col for col in df.columns if 'final_' in col or 'val' in col.lower() or 'accuracy' in col.lower()][:10])
print()

# Get complete training history in long format
print("=== New: Complete Training History (Long Format) ===")
history_df = sync_wandb_run.get_full_training_history()

if not history_df.empty:
    print("Shape:", history_df.shape)
    print("Columns:", list(history_df.columns))
    print("\nSample data:")
    # Show a few rows for one run and one metric
    sample = history_df[
        (history_df['run_id'] == history_df['run_id'].iloc[0]) & 
        (history_df['metric_name'] == history_df['metric_name'].iloc[0])
    ].head(5)
    display(sample)
else:
    print("No history data available in cache. Run sync first.")

In [ ]:
# NEW: Get training history in "unfolded" format (each epoch as separate column)
print("=== NEW: Unfolded Training History (Wide Format) ===")
print("Each epoch becomes a separate column: metric_epoch_0, metric_epoch_1, etc.")

# Get unfolded history for a few specific runs to keep it manageable
sample_run_ids = df['id'].head(3).tolist()  # Get first 3 run IDs
print(f"Unfolding history for runs: {sample_run_ids}")

unfolded_df = sync_wandb_run.get_unfolded_training_history(
    run_ids=sample_run_ids,
    metrics=['train_loss', 'val_loss', 'train_acc', 'val_acc', 'loss', 'accuracy']  # Specify metrics to avoid too many columns
)

if not unfolded_df.empty:
    print(f"Shape: {unfolded_df.shape}")
    print(f"Total columns: {len(unfolded_df.columns)}")
    
    # Show columns with epoch data
    epoch_columns = [col for col in unfolded_df.columns if '_epoch_' in col]
    print(f"Epoch columns (first 10): {epoch_columns[:10]}")
    
    # Show a sample with just basic info + some epoch columns
    sample_columns = ['run_id', 'run_name'] + epoch_columns[:8]  # First 8 epoch columns
    if len(sample_columns) > 2:
        print("\nSample data with epoch columns:")
        display(unfolded_df[sample_columns].head(2))
    
    # Show final values columns for comparison
    final_columns = [col for col in unfolded_df.columns if '_final' in col]
    if final_columns:
        print(f"\nFinal value columns: {final_columns}")
        sample_final = ['run_id', 'run_name'] + final_columns[:5]
        display(unfolded_df[sample_final].head(2))
else:
    print("No unfolded data available. Make sure runs are synced first.")

In [ ]:
# Practical example: Analyze convergence using unfolded format
if not unfolded_df.empty:
    print("=== Analysis Example: Early vs Late Performance ===")
    
    # Find loss columns across epochs
    loss_columns = [col for col in unfolded_df.columns if 'loss_epoch_' in col]
    acc_columns = [col for col in unfolded_df.columns if 'acc_epoch_' in col or 'accuracy_epoch_' in col]
    
    if loss_columns:
        print(f"Found {len(loss_columns)} loss epoch columns")
        
        # Compare early vs late performance
        early_loss = loss_columns[0] if loss_columns else None  # First epoch
        late_loss = loss_columns[-1] if len(loss_columns) > 5 else loss_columns[-1]  # Last epoch or 5th epoch
        
        if early_loss and late_loss and early_loss != late_loss:
            comparison_df = unfolded_df[['run_id', 'run_name', early_loss, late_loss]].copy()
            comparison_df['loss_improvement'] = comparison_df[early_loss] - comparison_df[late_loss]
            
            print(f"\nLoss improvement from {early_loss} to {late_loss}:")
            display(comparison_df)
    
    if acc_columns:
        print(f"\nFound {len(acc_columns)} accuracy epoch columns")
        print(f"Accuracy columns: {acc_columns[:5]}...")  # Show first 5
    
    print("\n=== Summary of Available Formats ===")
    print("1. Summary DataFrame (df): Final metrics + config - what you've been using")
    print("2. Long Format (history_df): Each metric-epoch as a row - good for plotting")  
    print("3. Wide Format (unfolded_df): Each epoch as a column - good for statistical analysis")
    print("\nChoose the format that best fits your analysis needs!")